In [1]:
# %%capture
# If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/accelerate-0.27.2-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/transformers-4.38.1-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/optimum-1.17.1-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/k/kevinchau321/making-wheels-of-necessary-packages-for-hf-llms/peft-0.9.0-py3-none-any.whl --find-links=/kaggle/input/k/kevinchau321/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/k/kevinchau321/making-wheels-of-necessary-packages-for-hf-llms/trl-0.8.0-py3-none-any.whl --find-links=/kaggle/input/k/kevinchau321/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/k/kevinchau321/making-wheels-of-necessary-packages-for-hf-llms/datasets-2.18.0-py3-none-any.whl --find-links=/kaggle/input/k/kevinchau321/making-wheels-of-necessary-packages-for-hf-llms

Looking in links: /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
Processing /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/bitsandbytes-0.42.0-py3-none-any.whl
Looking in links: /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
Processing /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/accelerate-0.27.2-py3-none-any.whl
accelerate is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Looking in links: /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
Processing /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/transformers-4.38.1-py3-none-any.whl
transformers is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Looking in links: /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
Processing /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/o

In [2]:
!wandb disabled

W&B disabled.


In [3]:
from accelerate.utils import BnbQuantizationConfig
from accelerate import Accelerator
import transformers
from transformers import TrainingArguments
import optimum
import bitsandbytes
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from datasets import Dataset

2024-03-22 21:53:14.688803: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 21:53:14.688904: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 21:53:14.865094: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import datetime
start_time = datetime.datetime.now()

In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

accelerator = Accelerator()

# Comment/Uncomment and use as per wish

# MODEL_PATH = "/kaggle/input/gemma/transformers/7b-it/2"
# MODEL_PATH = "/kaggle/input/gemma/transformers/2b-it/2"
# MODEL_PATH = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
MODEL_PATH = "/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1"
# MODEL_PATH = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"
# MODEL_PATH = "/kaggle/input/llama-2/pytorch/13b-chat-hf/1"

# Found a good blog to catch me up fast!
# https://huggingface.co/blog/4bit-transformers-bitsandbytes
# https://huggingface.co/docs/transformers/v4.38.1/en/quantization#compute-data-type
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)


model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config=quantization_config,
)

# model = model.to_bettertransformer()
model = accelerator.prepare(model)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [7]:
import pandas as pd
from tqdm import tqdm

DEBUG = False

# TEST_DF_FILE = '/kaggle/input/llm-prompt-recovery/test.csv'
TEST_DF_FILE = '/kaggle/input/combined/test2.csv'
SUB_DF_FILE = '/kaggle/input/llm-prompt-recovery/sample_submission.csv'
NROWS = None if DEBUG else None

if DEBUG:
    TEST_DF_FILE = '/kaggle/input/gemma-rewrite-nbroad/nbroad-v1.csv'
    SUB_DF_FILE = TEST_DF_FILE

tdf = pd.read_csv(TEST_DF_FILE, nrows=NROWS, usecols=['id', 'original_text', 'rewritten_text'])
sub = pd.read_csv(SUB_DF_FILE, nrows=NROWS, usecols=['id', 'rewrite_prompt'])

In [8]:
def truncate_txt(text, length):
    text_list = text.split()
    
    if len(text_list) <= length:
        return text
    
    return " ".join(text_list[:length])


def gen_prompt(og_text, rewritten_text):
    
    # Truncate the texts to first 200 words for now
    # As we are having memory issues on Mixtral8x7b
    og_text = truncate_txt(og_text, 200)
    rewritten_text = truncate_txt(rewritten_text, 200)
    
    return f"""Original Text:\"""{og_text}\"""\nRewritten Text:\"""{rewritten_text}\"""\nGiven are 2 texts, the Rewritten text was created from the Original text using the google Gemma model. You are trying to understand how the original text was transformed into a new version. Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten text. Start directly with the prompt, that's all I need. Output should be one line ONLY."""

# QLoRA Fine-tuning

In [9]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [10]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

In [11]:
# `LLM Prompt Recovery - Synthetic Datastore dataset` by @dschettler8845
df1 = pd.read_csv("/kaggle/input/llm-prompt-recovery-synthetic-datastore/gemma1000_w7b.csv")
df1 = df1[["original_text", "rewrite_prompt", "gemma_7b_rewritten_text_temp0"]]
df1 = df1.rename(columns={"gemma_7b_rewritten_text_temp0":"rewritten_text"})
df1.head(2)

# `3000 Rewritten texts - Prompt recovery Challenge` by @dipamc77
df2 = pd.read_csv("/kaggle/input/3000-rewritten-texts-prompt-recovery-challenge/prompts_0_500_wiki_first_para_3000.csv")
df2.head(2)

# Merge all datasets
df = pd.concat([df1, df2], axis=0)
df = df.sample(2000).reset_index(drop=True) # to reduce training time we are only using 2k samples
df.head(5)

,original_text,rewrite_prompt,rewritten_text
0,Enrique Eduardo Palos Reyes (born 31 May 1986 ...,Rewrite the text as if it were an epic poem ab...,"Sure, here's the rewritten text as if it were ..."
1,Jennifer Tress begins a presentation at George...,Express this as a lullaby.,"Sure, here is the lullaby in the form requeste..."
2,The Carl-Gustaf Rossby Research Medal is the h...,Style it as a passionate telenovela scene.,(The flickering candlelight casts a warm glow ...
3,Wallace M. Alexander (1869–1939) was an Americ...,Recast it as a survival guide for a deserted i...,"Sure, here's the recast:\n\nWallace M. Alexand..."
4,"The Guinea turaco (Tauraco persa), also known ...",Write the text as if it were a command issued ...,"Hear ye, hear ye, my dear subjects! I command ..."


In [12]:
dict = {
    "id": [str(x) for x in range(0,len(df))],
    "original_text": [str(x) for x in df.original_text],
    "rewrite_prompt": [str(x) for x in df.rewrite_prompt],
    "rewritten_text": [str(x) for x in df.rewritten_text],
}
dict['rewritten_text'][6]

"Sure, here is the arrangement to be arranged:\n\n**Arrange for the launch of the Tata 'Swach' water purifier in rural areas:**\n\n**Key points:**\n\n* The Tata Group has launched a new low-cost water purifier, the Tata 'Swach', aimed at lower-income households in rural areas.\n* The purifier is less than one metre tall and does not require running water or electricity to operate.\n* The firm is hoping to revolutionise the business of providing clean water, a lack of which affects almost one billion people globally.\n* The device is the result of a decade of research and development.\n\n**Steps to be arranged:**\n\n1. **Identify target areas:** Determine the specific rural areas where the Tata 'Swach' purifier will be most effective.\n2. **Distribution channels:** Set up a network of distribution channels to reach the target areas.\n3. **Training and education:** Train local communities on how to use and maintain the purifier.\n4. **Affordability:** Ensure that the purifier is affordab

In [13]:
ds = Dataset.from_dict(dict)
ds[0].keys()

dict_keys(['id', 'original_text', 'rewrite_prompt', 'rewritten_text'])

In [14]:
def truncate_text(text, max_len): #without this    
    text_split = text.split()    
    if len(text_split)<=max_len:
        return text
    else:
        return " ".join(text_split[:max_len])

def tokenize(example):
    for ot, rt, rp in zip(example["original_text"], example["rewritten_text"], example["rewrite_prompt"]):
        original_text = truncate_text(ot, 200)
        rewritten_text = truncate_text(rt, 200)
        rewrite_prompt = truncate_text(rp, 100)
    
        template = f"""Original Text:\"""{original_text}\"""\nRewritten Text:\"""{rewritten_text}\"""\nGiven are 2 texts, the Rewritten text was created from the Original text using the google Gemma model. You are trying to understand how the original text was transformed into a new version. Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten text. Start directly with the prompt, that's all I need. Output should be one line ONLY.\n\nResponse:\n\"{rewrite_prompt}\""""
        tkn = tokenizer(template, padding=True)
    return {**tkn}

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
ds_tokenize = ds.map(tokenize)
ds_tokenize[0].keys()

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

dict_keys(['id', 'original_text', 'rewrite_prompt', 'rewritten_text', 'input_ids', 'attention_mask'])

In [15]:
ds_split = ds_tokenize.train_test_split(test_size=0.20)

train_ds = ds_split['train']
test_ds = ds_split['test']

In [16]:
# # Setting sft parameters
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset,
#     peft_config=peft_config,
#     max_seq_length= None,
#     dataset_text_field="original_text",
#     tokenizer=tokenizer,
#     args=training_arguments,
#     packing= False,
# )

In [17]:
from transformers import Trainer, DataCollatorForLanguageModeling
trainer = Trainer(
    model=model, # lora enabled
    train_dataset=ds_tokenize,    
    args=TrainingArguments(
        per_device_train_batch_size=1,
        learning_rate=2e-5,
        fp16=True,
        output_dir=".",
        optim="paged_adamw_8bit",
        num_train_epochs=1
    ),
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    #compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [18]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
500,0.439500
1000,0.065200
1500,0.060600
2000,0.058400


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.

TrainOutput(global_step=2000, training_loss=0.15590352153778075, metrics={'train_runtime': 20139.6274, 'train_samples_per_second': 0.099, 'train_steps_per_second': 0.099, 'total_flos': 6.923214307472179e+16, 'train_loss': 0.15590352153778075, 'epoch': 1.0})

In [19]:
trainer.model.save_pretrained("My fine tuned model")

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(


# Test / Validate Fine-tuning

In [20]:
# test_data = pd.read_csv(open("/kaggle/input/combined/test2.csv"))
# test_data

In [21]:
# test_ds = Dataset.from_dict({
#     "id": [str(x) for x in range(0,len(df))],
#     "original_text": [x for x in df.original_text],
#     "rewritten_text": [x for x in df.rewritten_text],
# })
# test_ds

In [22]:
# def tokenize_test(example):
#     for ot, rt in zip(example["original_text"], example["rewritten_text"]):
#         original_text = truncate_text(ot, 200)
#         rewritten_text = truncate_text(rt, 200)
    
#         template = f"Instruction:\nGiven 2 texts: Original Text and Rewritten Text. An LLM is received a prompt from a user asking to rewrite the given Original Text. Based on this prompt, the LLM generated the given Rewritten Text from the given Original Text in a certain way specified in the prompt. Your task is to guess the prompt with which the LLM generated the Rewritten Text.\nOriginal Text: \"{original_text}\".\nRewritten Text: \"{rewritten_text}\".\n\nResponse:\n\"\""
#         tkn = tokenizer(template, padding=True, return_tensors="pt")
#     return {**tkn}
    
# test_ds_tokenized = test_ds.map(tokenize_test)
# test_ds_tokenized[0].keys()

In [23]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
# model = model.to(device)

# for item in test_ds:
#     print(item) # only 1 item so its okay to print out   
#     inputs = tokenize_test(item).to(device)
#     print(inputs)
    
#     #model.config.use_cache = False  # silence the warnings.
#     outputs = model.generate(**inputs)
#     print(outputs)

#     text_outputs = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     print('\n\n', text_outputs)


In [25]:
# text = "Improve the text"
# inputs = tokenizer(text, return_tensors="pt").to(device)
# print(inputs)

# model = model.to(device)
# model.config.use_cache = False  # silence the warnings.
# outputs = model.generate(**inputs)
# print(outputs)

# print('\n\n', tokenizer.decode(outputs[0], skip_special_tokens=True))

# Prediction

In [26]:
# TEST_DF_FILE = '/kaggle/input/testsmall/smalltest.csv'
TEST_DF_FILE = '/kaggle/input/llm-prompt-recovery/test.csv'
SUB_DF_FILE = '/kaggle/input/llm-prompt-recovery/sample_submission.csv'
NROWS = None if DEBUG else None

if DEBUG:
    TEST_DF_FILE = '/kaggle/input/gemma-rewrite-nbroad/nbroad-v1.csv'
    SUB_DF_FILE = TEST_DF_FILE

tdf = pd.read_csv(TEST_DF_FILE, nrows=NROWS, usecols=['id', 'original_text', 'rewritten_text'])

In [27]:
import gc
import re

device = accelerator.device
tdf['id'] = sub['id'].copy()

pbar = tqdm(total=tdf.shape[0])

it = iter(tdf.iterrows())
idx, row = next(it, (None, None))

# https://www.kaggle.com/competitions/llm-prompt-recovery/discussion/481116
DEFAULT_TEXT = "Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style."

res = []

while idx is not None:
    
    if (datetime.datetime.now() - start_time) > datetime.timedelta(hours=8, minutes=30):
        res.append([row["id"], DEFAULT_TEXT])
        idx, row = next(it, (None, None))
        pbar.update(1)
        continue
        
    torch.cuda.empty_cache()
    gc.collect()
        
    try:        
        messages = [
            {
                "role": "user",
                "content": gen_prompt(row["original_text"], row["rewritten_text"])
            }
        ]
        encoded_input = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(device)
        
        with torch.no_grad():
            encoded_output = model.generate(encoded_input, max_new_tokens=50, do_sample=True, pad_token_id=tokenizer.eos_token_id)
        
        decoded_output = tokenizer.batch_decode(encoded_output, skip_special_tokens=True)[0]
        decoded_output = result = re.sub(r"[\s\S]*\[\/INST\]", '', decoded_output, 1)
                
        res.append([row["id"], decoded_output])
                            
    except Exception as e:
        print(f"ERROR: {e}")
        res.append([row["id"], DEFAULT_TEXT])
        
    finally:
        idx, row = next(it, (None, None))
        pbar.update(1)

        
pbar.close()

  0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 1/1 [02:37<00:00, 157.71s/it]


In [28]:
# sub["rewrite_prompt"] = tdf['rewrite_prompt'].copy()
# sub.to_csv("submission.csv", index=False)
sub = pd.DataFrame(res, columns=['id', 'rewrite_prompt'])

sub.to_csv("sample_submission.csv", index=False)
sub.to_csv("submission.csv", index=False)

In [29]:
sub

,id,rewrite_prompt
0,9559194,"""""""""Rewrite original texts into a creative fo..."


In [30]:
res

[[9559194,
  ' """"Rewrite original texts into a creative format, using a clever prompt as your guide."""""]"']]